# Creating a logistic regression to predict absenteeism

## Import the relevant libraries

In [2]:
import pandas as pd
import numpy as np

## Load the data

In [3]:
data_preprocessed = pd.read_csv('Absenteeism_preprocessed_01.csv')

In [4]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,False,False,False,True,7,1,289,36,33,239.554,30,0,2,1,4
1,False,False,False,False,7,1,118,13,50,239.554,31,0,1,0,0
2,False,False,False,True,7,2,179,51,38,239.554,31,0,0,0,2
3,True,False,False,False,7,3,279,5,39,239.554,24,0,2,0,4
4,False,False,False,True,7,3,289,36,33,239.554,30,0,2,1,2


## Create the targets

In [5]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [6]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [7]:
data_preprocessed['Excessive Absenteeism'] = targets

In [8]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,False,False,False,True,7,1,289,36,33,239.554,30,0,2,1,4,1
1,False,False,False,False,7,1,118,13,50,239.554,31,0,1,0,0,0
2,False,False,False,True,7,2,179,51,38,239.554,31,0,0,0,2,0
3,True,False,False,False,7,3,279,5,39,239.554,24,0,2,0,4,1
4,False,False,False,True,7,3,289,36,33,239.554,30,0,2,1,2,0


## A comment on the targets

In [9]:
targets.sum() / targets.shape[0]

0.45571428571428574

In [10]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the Week',
                                            'Daily Work Load Average','Distance to Work'],axis=1)

In [11]:
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,False,False,False,True,7,289,33,30,0,2,1,1
1,False,False,False,False,7,118,50,31,0,1,0,0
2,False,False,False,True,7,179,38,31,0,0,0,0
3,True,False,False,False,7,279,39,24,0,2,0,1
4,False,False,False,True,7,289,33,30,0,2,1,0


## Select the inputs for the regression

In [12]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [15]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

## Standardize the data

In [13]:
from sklearn.preprocessing import StandardScaler

absenteeism_scaler = StandardScaler()

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        # self.scaler = StandardScaler(copy,with_mean,with_std)
        self.scaler = StandardScaler(copy=copy, with_mean=with_mean, with_std=with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [20]:
#columns_to_scale = ['Month Value','Day of the Week', 'Transportation Expense', 'Distance to Work',
       #'Age', 'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pet']

columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [21]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [22]:
absenteeism_scaler = CustomScaler(columns = columns_to_scale, copy=True,with_mean=True,with_std=True)

In [58]:
absenteeism_scaler.fit(unscaled_inputs);

d:\Download\archive\Anacond\Lib\site-packages\numpy\core\fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [59]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [60]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,False,False,False,True,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,False,False,False,False,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,False,False,False,True,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,True,False,False,False,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,False,False,False,True,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,True,False,False,False,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,True,False,False,False,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,True,False,False,False,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,False,False,False,True,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [61]:
scaled_inputs.shape

(700, 11)

## Split the data into train & test and shuffle

### Import the relevant module

In [62]:
from sklearn.model_selection import train_test_split

### Split

In [64]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, #train_size = 0.8, 
                                                                            test_size = 0.2, random_state = 20)

In [65]:
print (x_train.shape, y_train.shape)

(560, 11) (560,)


In [66]:
print (x_test.shape, y_test.shape)

(140, 11) (140,)


## Logistic regression with sklearn

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### Training the model

In [68]:
reg = LogisticRegression()

In [69]:
reg.fit(x_train,y_train)

LogisticRegression()

In [70]:
reg.score(x_train,y_train)

0.7732142857142857

### Manually check the accuracy

In [71]:
model_outputs = reg.predict(x_train)

In [72]:
np.sum((model_outputs==y_train))

433

In [73]:
np.sum((model_outputs==y_train)) / model_outputs.shape[0]

0.7732142857142857

### Finding the intercept and coefficients

In [74]:
reg.intercept_

array([-1.6469898])

In [75]:
reg.coef_

array([[ 2.80000644,  0.95174778,  3.1140605 ,  0.83835931,  0.15897713,
         0.60513709, -0.16990589,  0.27998236, -0.21017416,  0.34842434,
        -0.27721907]])

In [76]:
feature_name = unscaled_inputs.columns.values

In [ ]:
summary_table = pd.DataFrame (columns=['Feature name'], data = feature_name)

summary_table['Coefficient'] = np.transpose(reg.coef_)

,Feature name,Coefficient
0,Reason_1,2.800006
1,Reason_2,0.951748
2,Reason_3,3.114061
3,Reason_4,0.838359
4,Month Value,0.158977
5,Transportation Expense,0.605137
6,Age,-0.169906
7,Body Mass Index,0.279982
8,Education,-0.210174
9,Children,0.348424


In [78]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.646990
1,Reason_1,2.800006
2,Reason_2,0.951748
3,Reason_3,3.114061
4,Reason_4,0.838359
5,Month Value,0.158977
6,Transportation Expense,0.605137
7,Age,-0.169906
8,Body Mass Index,0.279982
9,Education,-0.210174


## Interpreting the coefficients

In [79]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [80]:
summary_table

,Feature name,Coefficient,Odds_ratio
0,Intercept,-1.646990,0.192629
1,Reason_1,2.800006,16.444753
2,Reason_2,0.951748,2.590233
3,Reason_3,3.114061,22.512270
4,Reason_4,0.838359,2.312570
5,Month Value,0.158977,1.172311
6,Transportation Expense,0.605137,1.831503
7,Age,-0.169906,0.843744
8,Body Mass Index,0.279982,1.323106
9,Education,-0.210174,0.810443


In [81]:
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature name,Coefficient,Odds_ratio
3,Reason_3,3.114061,22.512270
1,Reason_1,2.800006,16.444753
2,Reason_2,0.951748,2.590233
4,Reason_4,0.838359,2.312570
6,Transportation Expense,0.605137,1.831503
10,Children,0.348424,1.416833
8,Body Mass Index,0.279982,1.323106
5,Month Value,0.158977,1.172311
7,Age,-0.169906,0.843744
9,Education,-0.210174,0.810443


## Testing the model

In [82]:
reg.score(x_test,y_test)

0.75

In [83]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.71342516, 0.28657484],
       [0.5873216 , 0.4126784 ],
       [0.44016153, 0.55983847],
       [0.78163061, 0.21836939],
       [0.08407928, 0.91592072],
       [0.3348226 , 0.6651774 ],
       [0.29971206, 0.70028794],
       [0.13112385, 0.86887615],
       [0.78627908, 0.21372092],
       [0.74906578, 0.25093422],
       [0.49395555, 0.50604445],
       [0.22492002, 0.77507998],
       [0.07135527, 0.92864473],
       [0.73173354, 0.26826646],
       [0.30957854, 0.69042146],
       [0.54726422, 0.45273578],
       [0.55051921, 0.44948079],
       [0.53926379, 0.46073621],
       [0.40197149, 0.59802851],
       [0.05365482, 0.94634518],
       [0.70030387, 0.29969613],
       [0.78163061, 0.21836939],
       [0.42028246, 0.57971754],
       [0.42028246, 0.57971754],
       [0.24801464, 0.75198536],
       [0.74567806, 0.25432194],
       [0.51026557, 0.48973443],
       [0.8569309 , 0.1430691 ],
       [0.20365204, 0.79634796],
       [0.78163061, 0.21836939],
       [0.

### Logistic Regression Model Performance Summary
Training Accuracy (x_train):
The model achieved an accuracy of approximately 0.77, meaning it correctly classified 77% of training examples. This suggests that the model has learned meaningful patterns from the data.

### Testing Accuracy (x_test, y_test):
Using reg.score(x_test, y_test), the test set accuracy is approximately 0.75, indicating that 75% of unseen data points were correctly classified. This is a strong indication of generalization ability with only minimal overfitting (a drop of just 0.02 from training accuracy).

### Interpretation
The close match between training and testing accuracy shows that the model generalizes well.

The 0.75 testing accuracy is reasonable for a real-world classification task like absenteeism prediction, where data often contains noise and behavioral variance.

This result supports the model’s use for risk flagging, policy adjustment, or employee support targeting in HR contexts.

## Save the model

In [84]:
import pickle

In [85]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [86]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)